# Applied Data Science Capstone 
## Final Report
### To Locate a Suitable Place to OPEN a NEW SHOPPING MALL in BANGALORE, KARNATAKA, INDIA

##### 1. Build a dataframe of neighborhoods in Bangalore, Karnataka by web scraping the data from Wikipedia page
##### 2. Get the geographical coordinates of the neighborhoods
##### 3. Obtain the venue data for the neighborhoods from Foursquare API
##### 4. Explore and cluster the neighborhoods
##### 5. Select the best cluster to open a new shopping mall

###### ALSO PLEASE NOTE THAT REPORT AND PPT HAS BEEN UPLODED USING DIFFERENT LINK THIS IS ONLY THE NOTENOOK WHICH HAS THE PROGRAM PART

#### 1. IMPORTING LIBRARIES

In [70]:
import numpy as np # library to handle data 

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
import requests
#import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!pip install beautifulsoup4
#!pip install lxml
#!conda install -c conda-forge folium=0.5.0 --yes

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### 2. SCRAP DATA FROM WIKIPEDIA PAGE INTO DATAFRAME

In [71]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [72]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [73]:
# create a list to store neighborhood data
neighborhoodList = []

In [75]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [76]:
# create a new DataFrame from the list
bl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

bl_df.head()

,Neighborhood
0,"► Agara, Bangalore‎ (2 C, 6 F)"
1,► Arekere‎ (5 F)
2,"► Banashankari‎ (1 C, 4 F)"
3,► Banaswadi‎ (2 F)
4,"► Basavanagudi‎ (5 C, 11 F)"


In [77]:
# print the number of rows of the dataframe
bl_df.shape

(59, 1)

#### 3. GET THE GEOGRAPHICAL COORDINATES.

In [79]:
# define a function to get coordinates
def get_latlng(neighborhood):
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until WE get the coordinates
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, Karnataka'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [80]:
# call the function to get the coordinates, store in a new list using list comprehension
#!pip install geocoder
import geocoder
coords = [ get_latlng(neighborhood) for neighborhood in bl_df["Neighborhood"].tolist() ]

In [81]:
coords

[[12.841269941511115, 77.4815905253185],
 [12.997973409660101, 77.61047545115127],
 [12.90876015015293, 77.57288203719202],
 [12.997820409660104, 77.61038642561601],
 [12.938980000000072, 77.57137000000006],
 [12.882450000000063, 77.62475000000006],
 [12.927350000000047, 77.67185000000006],
 [12.981176497085812, 77.62506386955391],
 [12.966180000000065, 77.58690000000007],
 [12.958034857060447, 77.6009343090484],
 [12.817530000000033, 77.67879000000005],
 [12.966235877087087, 77.6067910877087],
 [12.855496201771626, 77.73111547336825],
 [13.250110000000063, 77.70788000000005],
 [12.997856409660098, 77.61040795115127],
 [12.943290000000047, 77.65602000000007],
 [12.839884329698537, 77.67221065989045],
 [12.998940000000061, 77.61276000000004],
 [12.942790000000059, 77.54122000000007],
 [13.047455598389988, 77.63327823980035],
 [13.049810000000036, 77.58903000000004],
 [12.957454844018699, 77.60090848251485],
 [12.912160000000029, 77.64490000000006],
 [12.978220000000022, 77.6339700000000

In [82]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [83]:
# merge the coordinates into the original dataframe
bl_df['Latitude'] = df_coords['Latitude']
bl_df['Longitude'] = df_coords['Longitude']

In [84]:
# check the neighborhoods and the coordinates
print(bl_df.shape)
bl_df.head()

(59, 3)


,Neighborhood,Latitude,Longitude
0,"► Agara, Bangalore‎ (2 C, 6 F)",12.841270,77.481591
1,► Arekere‎ (5 F),12.997973,77.610475
2,"► Banashankari‎ (1 C, 4 F)",12.908760,77.572882
3,► Banaswadi‎ (2 F),12.997820,77.610386
4,"► Basavanagudi‎ (5 C, 11 F)",12.938980,77.571370


In [85]:
# save the DataFrame as CSV file
bl_df.to_csv("bl_df.csv", index=False)

#### 4. CREATE A MAP OF BANGALORE WITH NEIGHBORHOODS SUPERIMPOSED ON TOP

In [86]:
# get the coordinates of Kuala Lumpur
address = 'Bangalore, Karnataka'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, Karnataka {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, Karnataka 12.9791198, 77.5912997.


In [32]:
# create map of Toronto using latitude and longitude values
map_bl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bl)  
    
map_bl

In [87]:
# save the map as HTML file
map_bl.save('map_kl.html')

#### 5. USE THE FOURSQUARE API TO EXPLORE THE NEIGHBORHOODS

In [88]:
# define Foursquare Credentials and Version
CLIENT_ID = 'GUMHEUVYSORG11WD5Z54PBV3TCWICMXYCVHMGCTGT0X10DSH' # your Foursquare ID
CLIENT_SECRET = '2TK4TPEZOETGMTFNJGUVA0RATOHNNACVTQ1BPGHXQLDSJXAF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GUMHEUVYSORG11WD5Z54PBV3TCWICMXYCVHMGCTGT0X10DSH
CLIENT_SECRET:2TK4TPEZOETGMTFNJGUVA0RATOHNNACVTQ1BPGHXQLDSJXAF


#### 6. Now, let's get the top 100 venues that are within a radius of 3000 meters

In [90]:
radius = 3000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [92]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4644, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Agara, Bangalore‎ (2 C, 6 F)",12.841270,77.481591,Art of Living International Center,12.844607,77.507343,Spiritual Center
1,"► Agara, Bangalore‎ (2 C, 6 F)",12.841270,77.481591,rachenamadu,12.850793,77.505317,Nature Preserve
2,"► Agara, Bangalore‎ (2 C, 6 F)",12.841270,77.481591,sнιvα sαι ιη∂υsтяү αη∂ тяα∂εяs,12.816927,77.491872,Outdoor Supply Store
3,► Arekere‎ (5 F),12.997973,77.610475,Mangalore Pearl - Seafood Restaurant,12.994472,77.615551,Seafood Restaurant
4,► Arekere‎ (5 F),12.997973,77.610475,Naturals Icecream,12.996912,77.611268,Ice Cream Shop


#### 7. Let's check how many venues were returned for each neighorhood

In [93]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Agara, Bangalore‎ (2 C, 6 F)",3,3,3,3,3,3
► Arekere‎ (5 F),100,100,100,100,100,100
► BEML‎ (7 F),100,100,100,100,100,100
"► Banashankari‎ (1 C, 4 F)",100,100,100,100,100,100
► Banaswadi‎ (2 F),100,100,100,100,100,100
"► Basavanagudi‎ (5 C, 11 F)",100,100,100,100,100,100
"► Begur, Bangalore‎ (1 C, 6 F)",32,32,32,32,32,32
"► Bellandur‎ (1 C, 4 F)",100,100,100,100,100,100
"► Bengaluru Pete‎ (9 C, 4 F)",100,100,100,100,100,100


#### 8. Let's find out how many unique categories can be curated from all the returned venues

In [94]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 195 uniques categories.


In [95]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Spiritual Center', 'Nature Preserve', 'Outdoor Supply Store',
       'Seafood Restaurant', 'Ice Cream Shop', 'Park', 'Tea Room',
       'Indian Restaurant', 'Department Store', 'Juice Bar', 'BBQ Joint',
       'Pub', 'Snack Place', 'Pakistani Restaurant',
       'Fast Food Restaurant', 'Steakhouse', 'Pizza Place',
       'Middle Eastern Restaurant', 'Café', 'Bakery',
       'Eastern European Restaurant', 'Afghan Restaurant',
       'Hyderabadi Restaurant', 'Shopping Mall', 'Concert Hall',
       "Women's Store", 'American Restaurant', 'Cricket Ground', 'Plaza',
       'Burger Joint', 'Bookstore', 'Lounge', 'Hotel', 'Coffee Shop',
       'Toy / Game Store', 'French Restaurant', 'Monument / Landmark',
       'Golf Course', 'Japanese Restaurant', 'Clothing Store',
       'Asian Restaurant', 'Andhra Restaurant', 'Chinese Restaurant',
       'Nightclub', 'Dessert Shop', 'Wine Shop', 'Electronics Store',
       'Hotel Bar', 'Bubble Tea Shop', 'Brewery'], dtype=object)

In [96]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

#### 9. Analyze Each Neighborhood

In [97]:
# one hot encoding
bl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

print(bl_onehot.shape)
bl_onehot.head()

(4644, 196)


,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nature Preserve,Neighborhood,Nightclub,North Indian Restaurant,Office,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Trail,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"► Agara, Bangalore‎ (2 C, 6 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Agara, Bangalore‎ (2 C, 6 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Agara, Bangalore‎ (2 C, 6 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [98]:
bl_grouped = bl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bl_grouped.shape)
bl_grouped

(59, 196)


,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nature Preserve,Neighborhood,Nightclub,North Indian Restaurant,Office,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Trail,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"► Agara, Bangalore‎ (2 C, 6 F)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.33

In [100]:
len(bl_grouped[bl_grouped["Shopping Mall"] > 0])

40

In [101]:
bl_mall = bl_grouped[["Neighborhoods","Shopping Mall"]]
bl_mall.head()

,Neighborhoods,Shopping Mall
0,"► Agara, Bangalore‎ (2 C, 6 F)",0.00
1,► Arekere‎ (5 F),0.03
2,► BEML‎ (7 F),0.02
3,"► Banashankari‎ (1 C, 4 F)",0.00
4,► Banaswadi‎ (2 F),0.03


#### 10. Cluster Neighborhoods

In [102]:
# set number of clusters
bclusters = 3

bl_clustering = bl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
#!pip install sklearn
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=bclusters, random_state=0).fit(bl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 0, 1, 2, 1, 1, 0, 0, 0], dtype=int32)

In [103]:
bl_merged = bl_mall.copy()

# add clustering labels
bl_merged["Cluster Labels"] = kmeans.labels_

In [104]:
bl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"► Agara, Bangalore‎ (2 C, 6 F)",0.00,1
1,► Arekere‎ (5 F),0.03,2
2,► BEML‎ (7 F),0.02,0
3,"► Banashankari‎ (1 C, 4 F)",0.00,1
4,► Banaswadi‎ (2 F),0.03,2


In [105]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bl_merged = bl_merged.join(bl_df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head() 

(59, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"► Agara, Bangalore‎ (2 C, 6 F)",0.00,1,12.841270,77.481591
1,► Arekere‎ (5 F),0.03,2,12.997973,77.610475
2,► BEML‎ (7 F),0.02,0,12.981176,77.625064
3,"► Banashankari‎ (1 C, 4 F)",0.00,1,12.908760,77.572882
4,► Banaswadi‎ (2 F),0.03,2,12.997820,77.610386


In [106]:
# sort the results by Cluster Labels
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"], inplace=True)
bl_merged

(59, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
29,► Kettohalli‎ (1 C),0.010000,0,12.957455,77.600908
27,"► Jayanagar, Bangalore‎ (1 C, 7 F)",0.010000,0,12.961606,77.597723
57,"► Yeshwantpur‎ (1 C, 6 F)",0.018519,0,13.029540,77.540220
31,► Konanakunte‎ (1 F),0.018868,0,12.890437,77.561763
35,"► Madiwala‎ (1 C, 6 F)",0.020000,0,12.956603,77.613550
36,"► Magadi‎ (2 C, 10 F)",0.020000,0,12.988227,77.605822
37,► Mahadevapura‎ (2 C),0.010000,0,12.958035,77.600934
38,► Majestic (Bangalore)‎ (1 C),0.010000,0,12.957455,77.600908
39,"► Malleswaram‎ (4 C, 2 F)",0.020000,0,12.995000,77.573460
40,"► Marathahalli‎ (8 C, 1 P, 30 F)",0.020619,0,12.955740,77.719419


#### 11. Finally, let's visualize the resulting clusters

In [107]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(bclusters)
ys = [i+x+(i*x)**2 for i in range(bclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude'], bl_merged['Longitude'], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [108]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### 12. Examine Clusters

#### 12.1 Cluster 0

In [109]:
bl_merged.loc[bl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
29,► Kettohalli‎ (1 C),0.010000,0,12.957455,77.600908
27,"► Jayanagar, Bangalore‎ (1 C, 7 F)",0.010000,0,12.961606,77.597723
57,"► Yeshwantpur‎ (1 C, 6 F)",0.018519,0,13.029540,77.540220
31,► Konanakunte‎ (1 F),0.018868,0,12.890437,77.561763
35,"► Madiwala‎ (1 C, 6 F)",0.020000,0,12.956603,77.613550
36,"► Magadi‎ (2 C, 10 F)",0.020000,0,12.988227,77.605822
37,► Mahadevapura‎ (2 C),0.010000,0,12.958035,77.600934
38,► Majestic (Bangalore)‎ (1 C),0.010000,0,12.957455,77.600908
39,"► Malleswaram‎ (4 C, 2 F)",0.020000,0,12.995000,77.573460
40,"► Marathahalli‎ (8 C, 1 P, 30 F)",0.020619,0,12.955740,77.719419


#### 12.2 Cluster 1

In [110]:
bl_merged.loc[bl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
48,"► Seetharampalya‎ (1 C, 14 F)",0.0,1,13.113200,77.424630
46,"► Ramamurthy Nagar‎ (1 C, 20 F)",0.0,1,13.023820,77.677850
45,"► Rajarajeshwari Nagar, Bangalore‎ (1 C, 4 F)",0.0,1,12.940380,77.541539
10,► Bommasandra‎ (33 F),0.0,1,12.817530,77.678790
12,► Chandapura‎ (4 F),0.0,1,12.855496,77.731115
13,"► Devanahalli‎ (5 C, 2 F)",0.0,1,13.250110,77.707880
3,"► Banashankari‎ (1 C, 4 F)",0.0,1,12.908760,77.572882
0,"► Agara, Bangalore‎ (2 C, 6 F)",0.0,1,12.841270,77.481591
16,"► Electronics City‎ (2 C, 34 F)",0.0,1,12.839884,77.672211
32,"► Koramangala‎ (1 C, 12 F)",0.0,1,12.935130,77.624450


#### 12.3 Cluster 2

In [111]:
bl_merged.loc[bl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
1,► Arekere‎ (5 F),0.030000,2,12.997973,77.610475
55,"► Whitefield, Bangalore‎ (4 C, 1 P, 22 F)",0.050000,2,12.975970,77.710317
24,► Ittamadu‎ (3 F),0.030000,2,12.997856,77.610408
14,► Dhobi Ghat (Bangalore)‎ (3 F),0.030000,2,12.997856,77.610408
17,"► Fraser Town, Bangalore‎ (1 C, 10 F)",0.030000,2,12.998940,77.612760
34,► Kundalahalli‎ (96 F),0.030000,2,12.967520,77.715000
33,"► Krishnarajapura‎ (3 C, 3 F)",0.030769,2,13.000390,77.683680
28,► Jeevanbheemanagar‎ (3 F),0.030000,2,12.997856,77.610408
4,► Banaswadi‎ (2 F),0.030000,2,12.997820,77.610386
11,"► Brigade Road, Bangalore‎ (3 C, 8 F)",0.030000,2,12.966236,77.606791


### CONCLUSION OR OBSERVATION

###### The results from the k-means clustering show that we can categorize the neighborhoods into 3 clusters based on the frequency of occurrence for “Shopping Mall”:

-  Cluster 0: Neighborhoods with moderate number of shopping malls
-  Cluster 1: Neighborhoods with low number to no existence of malls
-  Cluster 2: Neighborhoods with high concentration of shopping malls 

######  The results of the clustering are visualized in the map below with cluster 0 in red colour, cluster 1 in purple colour, and cluster 2 in mint green colour.
